# Coding Assignment 5

Group Members:
Zubair Lalani (zubairl2)
Adithya Swaminathan (adithya9)

## Setup

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'gap_statistic'

## Problem 1


Read the data and one-hot encode Gender predictor

In [3]:
DATA_PATH = "data/problem1/Mall_Customers.csv"
df = pd.read_csv(DATA_PATH)
df = df.rename(columns={"Genre": "Gender"}) # Fix misspelling of the column name in the original data
df.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [4]:
df = pd.get_dummies(df, columns=["Gender"])
df["Gender_Female"] = df["Gender_Female"].astype(int)
df["Gender_Male"] = df["Gender_Male"].astype(int)
df.head()

,CustomerID,Age,Annual Income (k$),Spending Score (1-100),Gender_Female,Gender_Male
0,1,19,15,39,0,1
1,2,21,15,81,0,1
2,3,20,16,6,1,0
3,4,23,16,77,1,0
4,5,31,17,40,1,0


In [5]:
df.dtypes

CustomerID                int64
Age                       int64
Annual Income (k$)        int64
Spending Score (1-100)    int64
Gender_Female             int64
Gender_Male               int64
dtype: object

Scale the features in our dataset and remove unnecessary columns

In [6]:
features_cols = ["Age", "Annual Income (k$)", "Spending Score (1-100)",
                "Gender_Female", "Gender_Male"]

X = df[features_cols] # removes CustomerID since we don't need it for our purposes
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### 1.A

In [8]:
sil_scores = {}
k_values = range(2, 11)

for k in k_values:
    # setting n_init to avoid a warning
    # set random_state for reproducibility
    # try diff k for n_clusters and compare results using sillhoutte score
    kmeans = KMeans(n_clusters=k, random_state=598, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    sil_scores[k] = score
    print(f"k = {k}, silhouette score = {score:.4f}")

# k with best silhouette
ksil = max(sil_scores, key=sil_scores.get)
print("Best k by silhouette:", ksil)

k = 2, silhouette score = 0.3964
k = 3, silhouette score = 0.3678
k = 4, silhouette score = 0.3322
k = 5, silhouette score = 0.3457
k = 6, silhouette score = 0.3621
k = 7, silhouette score = 0.3818
k = 8, silhouette score = 0.4016
k = 9, silhouette score = 0.4185
k = 10, silhouette score = 0.4378
Best k by silhouette: 10


In [10]:
def compute_wk(X, n_clusters, random_state=598):
    """Within-cluster dispersion (uses k-means inertia)."""
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10)
    kmeans.fit(X)
    return kmeans.inertia_


In [11]:
def gap_statistic(X, k_range, B=10, random_state=598):
    """
    X: data array (n_samples, n_features)
    k_range: iterable of k values to try, e.g. range(1, 11)
    B: number of reference datasets
    Returns:
        gaps: dict {k: Gap(k)}
        sdk: dict {k: s_k (std dev term)}
    """
    np.random.seed(random_state)
    n_samples, n_features = X.shape

    xmin = X.min(axis=0)
    xmax = X.max(axis=0)

    gaps = {}
    sdk = {}

    for k in k_range:
        # Wk for the real data
        wk = compute_wk(X, k, random_state=random_state)

        # Wk* for the reference datasets
        wk_refs = []
        for b in range(B):
            X_ref = np.random.uniform(xmin, xmax, size=(n_samples, n_features))
            wk_ref = compute_wk(X_ref, k, random_state=random_state + b + 1)
            wk_refs.append(wk_ref)

        wk_refs = np.array(wk_refs)
        log_wk_refs = np.log(wk_refs)

        # Gap(k) = mean(log(Wk*)) - log(Wk)
        gap_k = log_wk_refs.mean() - np.log(wk)
        gaps[k] = gap_k

        # Tibshirani's sd_k
        sk = np.sqrt(((log_wk_refs - log_wk_refs.mean()) ** 2).sum() / B)
        sdk[k] = sk * np.sqrt(1 + 1 / B)

        print(f"k = {k}, gap = {gap_k:.4f}")

    return gaps, sdk

In [12]:
k_range = range(1, 11)
gaps, sdk = gap_statistic(X_scaled, k_range, B=10, random_state=598)

k = 1, gap = -0.0401
k = 2, gap = 0.1279
k = 3, gap = 0.1669
k = 4, gap = 0.2244
k = 5, gap = 0.2499
k = 6, gap = 0.2835
k = 7, gap = 0.3643
k = 8, gap = 0.4796
k = 9, gap = 0.5531
k = 10, gap = 0.6432


In [13]:
kgap = max({k: v for k, v in gaps.items() if k > 1}, key=gaps.get)
print("kgap (max gap, k>1):", kgap)

kgap (max gap, k>1): 10


In [14]:
def plot_clusters(df, cluster_col, k, title_suffix):
    plt.figure(figsize=(7, 5))
    scatter = plt.scatter(
        df["Annual Income (k$)"],
        df["Spending Score (1-100)"],
        c=df[cluster_col],
        cmap="tab10"
    )
    plt.xlabel("Annual Income (k$)")
    plt.ylabel("Spending Score (1–100)")
    plt.title(f"K-Means Clusters (k = {k}) – {title_suffix}")
    plt.colorbar(scatter, label="Cluster")
    plt.show()

plot_clusters(df, "cluster_gap", kgap, "Gap Statistic k")
plot_clusters(df, "cluster_sil", ksil, "Silhouette k")

KeyError: 'cluster_gap'

<Figure size 700x500 with 0 Axes>

### 1.B

## Problem 2